In [2]:

%matplotlib auto
## TODO:
# refactorize to make it model -view- presenter
# pythonize slicing based on return value of spanSelector
# make xLabel of axis2 readable always
# add more info to show
# plot same day rectangles differently!
# update, linewidth, selectionColor on axis2
# adjust colors
# rotate labels,
# fix labelformat
# adjust spacing
# generalize button layout
# add functionality to change between log and lin scale

Using matplotlib backend: Qt5Agg


In [3]:
import os
import inspect
import datetime
from datetime import date, timedelta
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.widgets import Button, SpanSelector, RadioButtons
from matplotlib.dates import num2date, DayLocator

In [4]:
class Presenter():
    
    def __init__(self, model, view, files):
        self.model = model
        self.view = view
        self.files = files
        
    def showPlots(self):
        self.cleanedDf, self.wholeDf = self.model.parseFiles(self.files)
        self.view.showPlots(self.cleanedDf)
    
    def showDataframe(self):
        return self.wholeDf


In [5]:
class otpParser():
    
    def __init__(self):
        self.headers = ['accountNr', 'T/J', 'sum', 'currency', 'date', 'date2', 'currentValue', 'noIdea',
                   'noIdea2', 'comment', 'noIdea3', 'noIdea4', 'noIdea5']
        
    def parseFiles(self, files):
        dataFrames = [pd.read_csv(file, header=None) for file in files]
        mergedFrame = pd.concat(dataFrames)
        mergedFrame.columns = self.headers                                        
        mergedFrame = mergedFrame.reset_index(drop=True)
        mergedFrame = mergedFrame.sort_values(by='date')
        mergedFrame['comment'] = mergedFrame['comment'].replace(np.nan, '', regex=True)
        mergedFrame['comment'] = mergedFrame['comment'].apply((lambda x: ' '.join(x.split())))
        mergedFrame.loc[mergedFrame['comment']=='','comment'] = mergedFrame.loc[(mergedFrame["comment"] == '') , "noIdea5"]
        cleanDf = mergedFrame.loc[:,['sum', 'date', 'comment','currentValue']]
        return cleanDf, mergedFrame

    def printDf(self):
        return (self.mergedFrame.head())



In [ ]:
RED = (0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0)
DARK_RED = (0.49803921568627452, 0.12156862745098039, 0.12156862745098039, 1.0)
GREY = (0.5019607843137255, 0.5450980392156862, 0.5882352941176471, 1)
GREEN = (0.098039215686274508, 0.43529411764705883, 0.23921568627450981, 1.0)
DARK_GREEN = (0.0078431372549019607, 0.25490196078431371, 0.0078431372549019607, 1.0)
PURPLE = '#aaace2' # Color of the buttons, titles
LIGHT_GREEN =  (0.1568627450980392, 0.7058823529411765, 0.38823529411764707, 1)
LIGHT_RED = (1.0, 0.2784313725490196, 0.2784313725490196, 1)

LIGHT_GREY = '#d5d8dc'
width = 0.3
WIDTH = 12
G_RAT = (1 + 5 ** 0.5) / 2 # golden ratio
LABEL_ROTATION = 15 # DEGREES
DATEFORMATSTRING = '%Y-%m-%d'
DATEFROMAT = mdates.DateFormatter(DATEFORMATSTRING)
# to highlight recatangles
dark2light={DARK_RED:LIGHT_RED, DARK_GREEN:LIGHT_GREEN}
# to unhighlight recatangles
dark2normal={DARK_RED:RED, DARK_GREEN:GREEN}
light2normal={LIGHT_RED:RED, LIGHT_GREEN:GREEN}

    
class financeViewer():
    
    def __init__(self):

        self.box = dict(facecolor='blue', pad=3, alpha=0.2, boxstyle="Round4,pad=0.3")
        self.testString ="""Date: {}
                            Sum: {} HUF
                            Comment: {}"""
        self.scale1='log'
        self.scale2='log'
        
        self.start, self.end = None, None
    def connect2presenter(self, presenterObject):
        self.presenter = presenterObject
    
    def createFigure(self):
        print ('{} function is called'.format(inspect.stack()[0][3]))
        self.fig = plt.figure(figsize=(WIDTH, WIDTH/G_RAT),facecolor = LIGHT_GREY)

        self.gsp = gridspec.GridSpec(
                nrows = 3, ncols = 2, wspace = 0.05, hspace = 0.45,
                width_ratios = [G_RAT, 1], height_ratios = [(1+G_RAT)/G_RAT, G_RAT, 1])

        self.ax1 = plt.subplot(self.gsp[0,:])
        self.ax2 = plt.subplot(self.gsp[1:,0])
        self.ax3 = plt.subplot(self.gsp[1,1])
        self.ax4 = plt.subplot(self.gsp[2,1])

    def drawAxes(self):

        for ax in [self.ax1,self.ax2,self.ax3, self.ax4]: 
            ax.set_facecolor(GREY)
            
        #####BIG PLOT##       
        self.plotAx1()
        
        ####ZOOM PLOT##
        self.plotAx2()
        
        ##info plot##
        self.txt = self.ax3.text(0.1,0.5,'',
        horizontalalignment='left',
        verticalalignment='center',
        fontsize=12, color='black',
        wrap = True)
        self.ax3.set_xticks([]) 
        self.ax3.set_yticks([]) 
        self.ax3.set_title('info about the transactions', bbox=self.box)

        ### place of buttons##
        self.ax4.set_xticks([]) 
        self.ax4.set_yticks([]) 

    def on_plot_hover(self, event):
#         print ('{} function is called'.format(inspect.stack()[0][3]))
        if not event.inaxes: return
        if event.inaxes!= self.ax2: return

        for idx,bar in enumerate(self.ax2.patches):
            if bar.get_x() < event.xdata < bar.get_x() + bar.get_width():
                if bar.get_y() < event.ydata < bar.get_y() + bar.get_height(): 

                    self.ax2.patches[idx].set_facecolor(dark2light[bar.get_edgecolor()])
                    date_ordinal, y = self.ax2.transData.inverted().transform([event.x, event.y])+0.5
                    
                    # convert the numeric date into a datetime
                    transDate = num2date(date_ordinal).strftime(DATEFORMATSTRING)
                    pdDate = num2date(date_ordinal).strftime('%Y%m%d')
                    try:
                        comment = self.cleanDf.loc[(self.cleanDf['date'] == int(pdDate)) & (abs(self.cleanDf['sum'],)==bar.get_height()),'comment'].iloc[0]
                    except:
                        comment='Record not found'

                    newStr = self.testString.format(transDate,bar.get_height(), comment)
                    self.txt.set_text(newStr)
            else:
                self.ax2.patches[idx].set_facecolor(dark2normal[bar.get_edgecolor()])
        self.fig.canvas.draw()

    def reset_button_on_clicked(self, mouse_event):        
        self.plotAx2()

    def balanceView_button_on_clicked(self, mouse_event):
        self.txt.set_text('Not implemented yet')

    def transView_button_on_clicked(self, mouse_event):
        self.txt.set_text('Not implemented yet')

    def plotAx2(self, startDate=None, endDate=None):  
        self.ax2.cla()
        self.ax2.set_title('Selected duration', bbox=self.box)
        print ('start, end:', self.start, self.end)
        if self.start!=None:
#         if startDate!=None:
            zoomedTime = self.timeAxis[self.start:self.end]
            zoomedExpense = [expense[self.start: self.end] for expense in self.expense]
            zoomedIncome = self.income[self.start: self.end]
        else:
            zoomedTime = self.timeAxis
            zoomedExpense = self.expense
            zoomedIncome = self.income
        self.ax2.bar(zoomedTime, zoomedIncome, color=GREEN,edgecolor=DARK_GREEN)
        previousBars=copy.deepcopy(zoomedIncome)
        for expense in zoomedExpense:
            self.ax2.bar(zoomedTime, expense, bottom=previousBars,color=RED, edgecolor=DARK_RED)
            previousBars+=expense
                
        if startDate:
            self.ax2.xaxis.set_major_locator(DayLocator())
        self.ax2.xaxis.set_major_formatter(DATEFROMAT)
        self.ax2.set_yscale(self.scale2, nonposy='clip')
        self.ax2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
        plt.setp( self.ax2.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        
    def plotAx1(self):
        self.ax1.set_title('Whole duration',bbox=self.box)

        self.ax1.bar(self.timeAxis, self.income, color=GREEN,edgecolor=DARK_GREEN)
        previousBars=copy.deepcopy(self.income)
        for expense in self.expense:
            self.ax1.bar(self.timeAxis,expense, bottom=previousBars,color=RED, edgecolor=DARK_RED)
            previousBars+=expense

        self.span = SpanSelector(self.ax1, self.onselect, 'horizontal', 
                   rectprops=dict(alpha=0.3, facecolor=RED))
    
        plt.setp( self.ax1.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        self.ax1.xaxis.set_major_formatter(DATEFROMAT)
        self.ax1.set_yscale(self.scale1, nonposy='clip')
        self.ax1.yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))

    def onselect(self, xmin, xmax):
        
        dayMin, dayMax = sorted((int(xmin-0.5), int(xmax+0.5)))
        ##xmin, xmax is days from zero, if Xaxis is pandas daterange

        yearZero = datetime.datetime.strptime('0001/01/01', "%Y/%m/%d")
        startDate = yearZero + timedelta(days=dayMin)
        endDate = yearZero + timedelta(days=dayMax)
        st=str(startDate)[:10]
        nd=str(endDate)[:10]
        stIdx, = np.where( self.timeAxis.values==np.datetime64(st) )
        endIdx, = np.where( self.timeAxis.values==np.datetime64(nd) )

        try:
            stIdx , endIdx = stIdx[0], endIdx[0]
        except:
            try:
                stIdx , endIdx = 0, endIdx[0]
            except:
                stIdx , endIdx = stIdx[0], len(self.timeAxis)
        self.start, self.end = stIdx, endIdx
        self.plotAx2(stIdx, endIdx)
        self.fig.canvas.draw()
        
    def makeButtons(self):

        pos = self.ax4.get_position() # get the  position of axis ,which contains the buttons 
        rowNr, colNr = 1,2
        buttonwidth = 0.14
        buttonheight = 0.08
        Vspace = (pos.width - colNr*buttonwidth)/(colNr+1)
        Hspace = (pos.height - rowNr*buttonheight)/(rowNr+1)
        
        scaleSelectorAx = self.fig.add_axes([pos.x0+Vspace, pos.y0+Hspace, buttonwidth, buttonheight])
        viewSelectorAx = self.fig.add_axes([pos.x0+2*Vspace+buttonwidth, pos.y0+Hspace, buttonwidth, buttonheight])
        axcolor = PURPLE
        self.scaleSelector = RadioButtons(scaleSelectorAx, ('Ax1 linear', 'Ax2 linear', 'Ax1 logaritmic', 'Ax2 logaritmic'))
        self.viewSelector = RadioButtons(viewSelectorAx, ('transaction view', 'balance view'))
        
        for button in [self.scaleSelector, self.viewSelector]:
            for circle in button.circles: # adjust radius here. The default is 0.05
                circle.set_radius(0.08)
   
    def hzfunc(self, label):
            print (label)
            
    def scaleButtonClicked(self, label):
        print (label)
        if label == 'Ax1 linear':
            if self.scale1 == 'linear': return
            self.scale1='linear'
            self.plotAx1()
        elif label == 'Ax2 linear':
            if self.scale2 == 'linear': return
            self.scale2='linear'        
            self.plotAx2()

        elif label =='Ax1 logaritmic':
            if self.scale1 == 'log': return
            self.scale1='log'
            self.plotAx1()
        elif label == 'Ax2 logaritmic':
            if self.scale2 == 'log': return
            self.scale2='log'
            self.plotAx2()
        else: raise Error('label not supported')

        self.fig.canvas.draw()
        
    def connectButtons(self):

#         self.reset_button.on_clicked(self.reset_button_on_clicked)
        self.scaleSelector.on_clicked(self.scaleButtonClicked)
        self.viewSelector.on_clicked(self.hzfunc)

    def calculateDates(self):
        
        start = self.cleanDf['date'].values[0]
        end = self.cleanDf['date'].values[-1]
        self.pandasStartDate = "{0}-{1}-{2}".format(str(start)[0:4], str(start)[4:6], str(start)[6:8])
        self.startDate = date(int(str(start)[0:4]), int(str(start)[4:6]), int(str(start)[6:8]))
        self.endDate = date(int(str(end)[0:4]), int(str(end)[4:6]), int(str(end)[6:8]))
        self.timeDelta = self.endDate - self.startDate        
        
    def calculateAxes(self):
        self.calculateDates()
        self.separateTransactions()
        self.timeAxis = pd.date_range(self.pandasStartDate, periods=self.timeDelta.days+1, freq='D')

    def separateTransactions(self):

        days = [int((self.startDate + timedelta(days=i)).strftime('%Y%m%d')) for i in range(self.timeDelta.days + 1)]
        transDays = self.cleanDf['date'].values
        heights = self.cleanDf['sum'].values
        expense = [-figure if figure<0 else 0 for figure in heights]
        income = [figure if figure>0 else 0 for figure in heights]
        Xs = transDays

        values, counts = np.unique(Xs, return_counts=True)
        WholeY=[]
        smallY=np.zeros_like(days)  
        payment = np.zeros_like(days)

        for freq in range(1,max(counts)+1): 
            for val, cnt in zip(values, counts):
                if cnt >= freq:
                    index = np.where(Xs==val)[0][freq-1]
                    if heights[index] > 0:
                        payment [days.index(val)] = heights[index]
                    else:
                        smallY[days.index(val)] = -heights[index]
            WholeY.append(smallY)
            smallY=np.zeros_like(days)  

        self.expense = WholeY
        self.income = payment
    
    def showPlots(self, cleanDf):
        self.cleanDf = cleanDf
        self.calculateAxes()
        self.createFigure()
        self.drawAxes()

        self.fig.canvas.mpl_connect('button_press_event', self.on_plot_hover) 
        self.fig.subplots_adjust(left=0.06, bottom=0.07, right=0.97, top=0.95)
        self.makeButtons()
        self.connectButtons()

        plt.show()

In [ ]:


files =  [file for file in os.listdir() if file.lower().endswith('csv')]
model = otpParser()
view = financeViewer()


myApp = Presenter(model, view, files)

In [ ]:
myApp.showPlots()

In [ ]:

# viewer = financeViewer()
# viewer.showPlots(myApp.timeAxis ,myApp.transactions,myApp.showDataframe())


In [ ]:

files =  [file for file in os.listdir() if file.lower().endswith('csv')]
print(files)

In [ ]:

file = '2017april.csv'
apr=pd.read_csv(file, header=None)
file = '2017may.csv'
may=pd.read_csv(file, header=None)
file = '2017june.csv'
june=pd.read_csv(file, header=None)

In [ ]:
apr.shape
# apr.head(2)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

x = np.linspace(0, 4*np.pi, 100)
fig, axes = plt.subplots(nrows=3)

lines = [axes[0].plot(x, np.cos(x), animated=True)[0]]
lines += [ax.plot(x, np.cos(x)) for ax in axes[1:]]

class Update(object):
    def __init__(self, line):
        self.phase = 0
        self.line = line
    def __call__(self, _):
        self.line.set_ydata(np.cos(x + self.phase / 5.0))
        self.phase += 1.0
        return [self.line]

anim = FuncAnimation(fig, Update(lines[0]), interval=0, blit=True) 
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.ion()
x = np.linspace(0, 4*np.pi, 100)
fig, axes = plt.subplots(nrows=3)

fig.canvas.draw()
background = fig.canvas.copy_from_bbox(axes[0].bbox)

lines = [ax.plot(x, np.cos(x))[0] for ax in axes]
fig.canvas.draw()

for phase in range(1000):
    fig.canvas.restore_region(background)
    lines[0].set_ydata(np.cos(x + phase / 5.0))
    axes[0].draw_artist(lines[0])
    fig.canvas.blit(axes[0].bbox)

In [ ]:
june.shape

In [ ]:
df=myApp.showDataframe()
df['comment2'] = df['comment']

In [ ]:
df.head(15)

In [ ]:
from pylab import *
import matplotlib.pyplot  as pyplot
a = [ pow(10,i) for i in range(10) ]
fig = pyplot.figure()
ax = fig.add_subplot(2,1,1)

line, = ax.plot(a, color='blue', lw=2)

ax.set_yscale('log')
ax.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%d'))
show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RadioButtons

t = np.arange(0.0, 2.0, 0.01)
s0 = np.sin(2*np.pi*t)
s1 = np.sin(4*np.pi*t)
s2 = np.sin(8*np.pi*t)

fig, ax = plt.subplots()
l, = ax.plot(t, s0, lw=2, color='red')
plt.subplots_adjust(left=0.3)

axcolor = 'lightgoldenrodyellow'
rax = plt.axes([0.05, 0.7, 0.15, 0.15], facecolor=axcolor)
radio = RadioButtons(rax, ('2 Hz', '4 Hz', '8 Hz'))


def hzfunc(label):
    hzdict = {'2 Hz': s0, '4 Hz': s1, '8 Hz': s2}
    ydata = hzdict[label]
    l.set_ydata(ydata)
    plt.draw()
radio.on_clicked(hzfunc)

rax = plt.axes([0.05, 0.4, 0.15, 0.15], facecolor=axcolor)
radio2 = RadioButtons(rax, ('red', 'blue', 'green'))


def colorfunc(label):
    l.set_color(label)
    plt.draw()
radio2.on_clicked(colorfunc)

rax = plt.axes([0.05, 0.1, 0.15, 0.15], facecolor=axcolor)
radio3 = RadioButtons(rax, ('-', '--', '-.', 'steps', ':'))


def stylefunc(label):
    l.set_linestyle(label)
    plt.draw()
radio3.on_clicked(stylefunc)

plt.show()

In [ ]:
"""
The SpanSelector is a mouse widget to select a xmin/xmax range and plot the
detail view of the selected region in the lower axes
"""
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import SpanSelector

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(211, facecolor='#FFFFCC')

x = np.arange(0.0, 5.0, 0.01)
y = np.sin(2*np.pi*x) + 0.5*np.random.randn(len(x))

ax.bar(np.arange(200), np.random.rand(200))
ax.set_ylim(-2, 2)
ax.set_title('Press left mouse button and drag to test')

ax2 = fig.add_subplot(212, facecolor='#FFFFCC')
line2, = ax2.plot(x, y, '-')


def onselect(xmin, xmax):
    indmin, indmax = np.searchsorted(x, (xmin, xmax))
    indmax = min(len(x) - 1, indmax)

    thisx = x[indmin:indmax]
    thisy = y[indmin:indmax]
    line2.set_data(thisx, thisy)
    ax2.set_xlim(thisx[0], thisx[-1])
    ax2.set_ylim(thisy.min(), thisy.max())
    fig.canvas.draw()

# set useblit True on gtkagg for enhanced performance
span = SpanSelector(ax, onselect, 'horizontal', useblit=True,
                    rectprops=dict(alpha=0.5, facecolor='red'))


plt.show()